In [ ]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew

In [ ]:
base_directory = "../results/time_series/"
#results_column = "mse"
results_column = "mae"
round_digits = 4
cvar_alpha = 0.95

In [ ]:
experiments_by_horizon = [
["Autoformer_ETTh1_96_96_0_100.csv", "iTransformer_ETTh1_96_96_0_100.csv", "NLinear_ETTh1_96_96_0_100.csv", "TSMixer_ETTh1_96_96_0_100.csv"],
["Autoformer_ETTh1_96_192_0_100.csv", "iTransformer_ETTh1_96_192_0_100.csv", "NLinear_ETTh1_96_192_0_100.csv", "TSMixer_ETTh1_96_192_0_100.csv"],
["Autoformer_ETTh1_96_336_0_100.csv", "iTransformer_ETTh1_96_336_0_100.csv", "NLinear_ETTh1_96_336_0_100.csv", "TSMixer_ETTh1_96_336_0_100.csv"],
["Autoformer_ETTh1_96_720_0_100.csv", "iTransformer_ETTh1_96_720_0_100.csv", "NLinear_ETTh1_96_720_0_100.csv", "TSMixer_ETTh1_96_720_0_100.csv"],

["Autoformer_ETTh2_96_96_0_100.csv", "iTransformer_ETTh2_96_96_0_100.csv", "NLinear_ETTh2_96_96_0_100.csv", "TSMixer_ETTh2_96_96_0_100.csv"],
["Autoformer_ETTh2_96_192_0_100.csv", "iTransformer_ETTh2_96_192_0_100.csv", "NLinear_ETTh2_96_192_0_100.csv", "TSMixer_ETTh2_96_192_0_100.csv"],
["Autoformer_ETTh2_96_336_0_100.csv", "iTransformer_ETTh2_96_336_0_100.csv", "NLinear_ETTh2_96_336_0_100.csv", "TSMixer_ETTh2_96_336_0_100.csv"],
["Autoformer_ETTh2_96_720_0_100.csv", "iTransformer_ETTh2_96_720_0_100.csv", "NLinear_ETTh2_96_720_0_100.csv", "TSMixer_ETTh2_96_720_0_100.csv"],

["Autoformer_ETTm1_96_96_0_100.csv", "iTransformer_ETTm1_96_96_0_100.csv", "NLinear_ETTm1_96_96_0_100.csv", "TSMixer_ETTm1_96_96_0_100.csv"],
["Autoformer_ETTm1_96_192_0_100.csv", "iTransformer_ETTm1_96_192_0_100.csv", "NLinear_ETTm1_96_192_0_100.csv", "TSMixer_ETTm1_96_192_0_100.csv"],
["Autoformer_ETTm1_96_336_0_100.csv", "iTransformer_ETTm1_96_336_0_100.csv", "NLinear_ETTm1_96_336_0_100.csv", "TSMixer_ETTm1_96_336_0_100.csv"],
["Autoformer_ETTm1_96_720_0_100.csv", "iTransformer_ETTm1_96_720_0_100.csv", "NLinear_ETTm1_96_720_0_100.csv", "TSMixer_ETTm1_96_720_0_100.csv"],

["Autoformer_ETTm2_96_96_0_100.csv", "iTransformer_ETTm2_96_96_0_100.csv", "NLinear_ETTm2_96_96_0_100.csv", "TSMixer_ETTm2_96_96_0_100.csv"],
["Autoformer_ETTm2_96_192_0_100.csv", "iTransformer_ETTm2_96_192_0_100.csv", "NLinear_ETTm2_96_192_0_100.csv", "TSMixer_ETTm2_96_192_0_100.csv"],
["Autoformer_ETTm2_96_336_0_100.csv", "iTransformer_ETTm2_96_336_0_100.csv", "NLinear_ETTm2_96_336_0_100.csv", "TSMixer_ETTm2_96_336_0_100.csv"],
["Autoformer_ETTm2_96_720_0_100.csv", "iTransformer_ETTm2_96_720_0_100.csv", "NLinear_ETTm2_96_720_0_100.csv", "TSMixer_ETTm2_96_720_0_100.csv"],

["Autoformer_traffic_96_96_0_100.csv", "iTransformer_traffic_96_96_0_100.csv", "NLinear_traffic_96_96_0_100.csv", "TSMixer_traffic_96_96_0_100.csv"],
["Autoformer_traffic_96_192_0_100.csv", "iTransformer_traffic_96_192_0_100.csv", "NLinear_traffic_96_192_0_100.csv", "TSMixer_traffic_96_192_0_100.csv"],
["Autoformer_traffic_96_336_0_100.csv", "NLinear_traffic_96_336_0_100.csv", "TSMixer_traffic_96_336_0_100.csv"],
["Autoformer_traffic_96_720_0_100.csv", "NLinear_traffic_96_720_0_100.csv", "TSMixer_traffic_96_720_0_100.csv"],

["Autoformer_weather_96_96_0_100.csv", "iTransformer_weather_96_96_0_100.csv", "NLinear_weather_96_96_0_100.csv", "TSMixer_weather_96_96_0_100.csv"],
["Autoformer_weather_96_192_0_100.csv", "iTransformer_weather_96_192_0_100.csv", "NLinear_weather_96_192_0_100.csv", "TSMixer_weather_96_192_0_100.csv"],
["Autoformer_weather_96_336_0_100.csv", "iTransformer_weather_96_336_0_100.csv", "NLinear_weather_96_336_0_100.csv", "TSMixer_weather_96_336_0_100.csv"],
["Autoformer_weather_96_720_0_100.csv", "iTransformer_weather_96_720_0_100.csv", "NLinear_weather_96_720_0_100.csv", "TSMixer_weather_96_720_0_100.csv"],
]

In [ ]:
def calculate_cvar(dataset, alpha):
    # alpha = 0.90 = 90% 
    # alpha = 0.95 = 95%
    # alpha = 0.99 = 99%    

    dataset.sort()
    # Reverse the list
    dataset = dataset[::-1]
    var = np.quantile(dataset, alpha)
    cvar = dataset[dataset >= var].mean().round(4)
    return(cvar)

In [ ]:
def save_kde(data, title):
    fig, ax = plt.subplots(figsize=(20, 10))

    sns.histplot(
        data,
        kde=True,
        stat="proportion",
        kde_kws=dict(cut=3),
        legend=True,
    )

    plt.title(title, fontsize=16)

    plt.show()

    fig.savefig(title.replace(":", "").replace(" ",  "_") + "_kde.png", pad_inches=0.1, bbox_inches='tight')

In [ ]:
def save_boxplot(data, title, ylabel):
    fig, ax = plt.subplots(figsize=(15, 7))

    sns.boxplot(pd.DataFrame.from_dict(data))

    plt.title(title, fontsize=16)
    plt.ylabel(ylabel.upper(), fontsize=16)
    plt.tick_params(labelsize=14)

    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.set_xticklabels(ax.get_xticklabels(),rotation=90)

    plt.show()

    fig.savefig(title.replace(":", "").replace(" ",  "_") + "_box.png", pad_inches=0.1, bbox_inches='tight')

In [ ]:
def save_cvar(data, title):

    # Calculate the min and max values for the x-axis
    min = 0
    max = 0

    for run_name in data:
        run_min = data[run_name].min()
        run_max = data[run_name].max()
    
        if run_min < min or min == 0:
            min = run_min
        if run_max > max or max == 100:
            max = run_max

    min = min - (min * 0.05)
    max = max + (max * 0.05)

    # Set the number of columns and rows for the subplots
    # 2 columns and as many rows as needed to fit all the data
    ncols = 2
    nrows = math.ceil((len(data) / ncols))

    if nrows == 1:
        nrows = 2

    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(20, 10))
    fig.tight_layout(pad=3)

    for idx, run_name in enumerate(data):
        # Find the row and column for the current plot
        column = idx % ncols
        row = idx // ncols

        # Calculate the summary statistics
        dataset_mean = data[run_name].mean()
        cvar = calculate_cvar(data[run_name], cvar_alpha)

        sns.histplot(
            data[run_name],
            kde=True,
            kde_kws=dict(cut=3),
            legend=True,
            ax=ax[row, column],
        ).set_title(run_name)

        # Add the mean and CVaR to the plot
        ax[row, column].axvline(dataset_mean, color='red', linestyle='solid', label="Mean: %.2f%% " % (dataset_mean *100) )
        ax[row, column].axvline(cvar, color='red', linestyle='dashed', label="CVaR: %.2f%% " % (cvar *100)  )

        ax[row, column].set(ylabel='')
        ax[row, column].tick_params(left=False, bottom=True)
        ax[row, column].axes.get_yaxis().set_visible(False)

        ax[row, column].legend()
        ax[row, column].set(xlabel="Accuracy")
        ax[row, column].set_xlim([min, max])
        ax[row, column].set_ylim([0, 30])

    plt.show()

    fig.savefig(title.replace(":", "").replace(" ",  "_") + "_cvar.png", pad_inches=0.1, bbox_inches='tight')

In [ ]:
# Save the summary statistics so they can be saved to a CSV file
summary_statistics = []

for idx, experiments in enumerate(experiments_by_horizon):

    title = ""

    horizon = experiments[1].split("_")[3]

    # Save the individual results for the plots
    results_values = {}

    # Loop through the individual experiments
    for experiment in experiments:

        df = pd.read_csv(base_directory + experiment)

        # Get the 100 results
        results = df[results_column].values
        # Calculate the summary statistics
        mean = np.mean(results).round(round_digits)
        median = np.median(results).round(round_digits)
        min = np.min(results).round(round_digits)
        max = np.max(results).round(round_digits)
        std = np.std(results).round(round_digits)
        cvar = calculate_cvar(results, cvar_alpha)
        skewness = skew(results).round(round_digits)

        # Get the model and dataset from the experiment name
        dataset = experiment.split("_")[1]
        model = experiment.split("_")[0]

        # Add the dataset to the title if it is not already there
        if dataset not in title:
            title = dataset + " " + horizon + " " + results_column.upper()

        # Save the results for the plot
        results_values[model] = results

        # Save the summary statistics
        summary_statistics.append([dataset, model, horizon, mean, median, min, max, std, cvar, skewness])

    # Plot the results for the experiment as a KDE histogram
    save_kde(results_values, title)
    # Plot the results for the experiment as a boxplot
    save_boxplot(results_values, title, results_column)
    # Plot the results for the experiment as a CVaR histogram
    save_cvar(results_values, title)

In [ ]:
summary_statistics_df = pd.DataFrame(summary_statistics, columns=["Dataset", "Model", "Horizon", "Mean", "Median", "Min", "Max", "Std", "CVaR 95%", "Skewness"])
summary_statistics_df.to_csv("time_series_summary_statistics_" + results_column + ".csv", index=False)